In [2]:
import pickle
import pandas as pd
import torch
import numpy as np

In [3]:
mimiciv_preprocessed_path = "/home/hhchung/MIMIC-IV-Data-Pipeline/XY_dl_data.pkl"
mimiciv_ditems_path = "/mnt/d64c1162-08cc-4571-90a3-04c60b6f6f66/hhchung/mimic-code/data/mimiciv/3.1/icu/d_items.csv.gz"

In [4]:
ditems = pd.read_csv(mimiciv_ditems_path, compression="gzip")

In [5]:
# make a dictionary of itemid to label
itemid_to_label = dict(zip(ditems["itemid"], ditems["label"]))

In [6]:
ditems.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [7]:
mimiciv = pickle.load(open(mimiciv_preprocessed_path, "rb"))

## Filter By Column

In [8]:
# find the dimensions/features/columns of the meds time series with the most non-zero values
med_column_idx_sorted_by_nonzero_vals = torch.argsort((mimiciv["meds"] != 0).sum(dim=[0, 1]), descending=True)

num_selected_cols = 10
for column_idx in med_column_idx_sorted_by_nonzero_vals[:num_selected_cols]:
    column_num = int(mimiciv["keys_to_cols"]["MEDS"][column_idx])
    column_name = itemid_to_label[column_num]
    batch_timeseries = mimiciv["meds"][:, :, column_idx]
    nonzero_ratio = (batch_timeseries != 0).sum(dim=[0, 1]) / batch_timeseries.shape[0] / batch_timeseries.shape[1]
    print(column_num, column_name, round(nonzero_ratio.item(), 3))

    

225158 NaCl 0.9% 0.353
220949 Dextrose 5% 0.222
225152 Heparin Sodium 0.089
225943 Solution 0.065
222168 Propofol 0.046
225828 LR 0.041
225823 D5 1/2NS 0.032
228340 Furosemide (Lasix) 250/50 0.021
223258 Insulin - Regular 0.021
225936 Replete with Fiber (Full) 0.018


In [9]:
# find the dimensions/features/columns of the chart time series with the most non-zero values
chart_column_idx_sorted_by_nonzero_vals = torch.argsort((mimiciv["chart"] != 0).sum(dim=[0, 1]), descending=True)

num_selected_cols = 20
for column_idx in chart_column_idx_sorted_by_nonzero_vals[:num_selected_cols]:
    column_num = int(mimiciv["keys_to_cols"]["CHART"][column_idx])
    column_name = itemid_to_label[column_num]
    batch_timeseries = mimiciv["chart"][:, :, column_idx]
    nonzero_ratio = (batch_timeseries != 0).sum(dim=[0, 1]) / batch_timeseries.shape[0] / batch_timeseries.shape[1]
    print(column_num, column_name, round(nonzero_ratio.item(), 3))

    

220045 Heart Rate 0.999
220277 O2 saturation pulseoxymetry 0.999
220046 Heart rate Alarm - High 0.997
220047 Heart Rate Alarm - Low 0.997
220210 Respiratory Rate 0.997
223770 O2 Saturation Pulseoxymetry Alarm - Low 0.996
224161 Resp Alarm - High 0.996
223769 O2 Saturation Pulseoxymetry Alarm - High 0.996
224162 Resp Alarm - Low 0.996
223761 Temperature Fahrenheit 0.995
226253 SpO2 Desat Limit 0.995
220645 Sodium (serum) 0.992
227442 Potassium (serum) 0.992
220602 Chloride (serum) 0.992
225624 BUN 0.992
227443 HCO3 (serum) 0.992
220621 Glucose (serum) 0.992
227073 Anion gap 0.992
220545 Hematocrit (serum) 0.99
220635 Magnesium 0.99


### Asked ChatGPT o3 does any of the medications have causal effects on any of the chart events
* Regular insulin
    * Glucose (serum)
    * Potassium (serum)
* Furosemide (Lasix)
    * Potassium (serum)
    * Magnesium (serum)
    * BUN
* 0.9% Sodium Chloride
    * Sodium (serum)
    * Chloride (serum)
* 5% Dextrose in water (D5W)
    * Glucose (serum)
    * Sodium (serum)
* Heparin (continuous />= 4 days)
    * Potassium (serum)
* Propofol infusion
    * Respiratory rate
    * O2 Saturation
* Lactated Ringer's (LR)
    * Bicarbonate
    * Potassium

Therefore, we pick the following chart events:
* Glucose (serum)
* Potassium (serum)
* Magnesium (serum)
* BUN

and the following medications:
* Regular insulin
* Furosemide (Lasix)

In [10]:
selected_med_column_nums = [223258, # insulin
                            228340, # furosemide (lasix)
                            # 225158, # 0.9% sodium chloride,
                            # 220949, # 5% dextrose in water (d5w)
                            # 225152, # heparin (continuous >= 4 days)
                            # 222168, # propofol infusion
                            # 225828, # lactated ringer's (lr)
                            ]

selected_chart_column_nums = [220621, # glucose (serum)
                              227442, # potassium (serum)
                              220635, # magnesium (serum)
                              225624, # BUN
                            #   220645, # sodium (serum)
                            #   220602, # chloride (serum)
                            #   220210, # respiratory rate
                            #   220277, # o2 saturation
                            #   227443, # bicarbonate (hco3)
                            #   227442, # pottasium (serum)
                              ]



## Check MIMIC-IV RUS Results
**Q**: Which ones of the following have a stronger effect on a person's Potassium level few hours later? How would you rank them: Insulin medication, Furosemide medication, current Glucose level, current Potassium level, current Magnesium level, and current Blood Urea Nitrogen (BUN)?


**A (ChatGPT o3)**:

Quick ranking for their influence on serum potassium over the next few hours
(strongest → weakest, assuming an otherwise-stable adult with normal kidneys and acid-base status):

1. Insulin medication

2. Current serum potassium concentration itself

3. Furosemide (loop-diuretic) dose

4. Current glucose level

5. Current magnesium level

6. Current blood urea nitrogen (BUN)







In [11]:
mimiciv_rus_results = np.load("../results/mimiciv/mimiciv_lag6_bins8_thresh0.3.npy", allow_pickle=True)


In [12]:
mimiciv_rus_results[0:6]

array([{'feature_pair': ('Insulin - Regular', 'Furosemide (Lasix) 250/50'), 'lag': 1, 'dominant_term': 'U1', 'R_value': np.float64(0.05823217651439122), 'U1_value': np.float64(0.16768299611863904), 'U2_value': np.float64(3.483038856699689e-08), 'S_value': np.float64(0.16398752686165957), 'MI_value': np.float64(0.38990273432507866)},
       {'feature_pair': ('Insulin - Regular', 'Furosemide (Lasix) 250/50'), 'lag': 2, 'dominant_term': 'U1', 'R_value': np.float64(0.04292269726383163), 'U1_value': np.float64(0.20691338251675262), 'U2_value': np.float64(4.8013916188512935e-08), 'S_value': np.float64(0.14777069391400488), 'MI_value': np.float64(0.39760682170850503)},
       {'feature_pair': ('Insulin - Regular', 'Furosemide (Lasix) 250/50'), 'lag': 3, 'dominant_term': 'U1', 'R_value': np.float64(0.02977073397945873), 'U1_value': np.float64(0.3097256486023837), 'U2_value': np.float64(4.736179194982763e-08), 'S_value': np.float64(0.13581835170862006), 'MI_value': np.float64(0.4753147816522544

In [13]:
mimiciv_rus_results[6:12]

array([{'feature_pair': ('Insulin - Regular', 'Glucose (serum)'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.20780064333731002), 'U1_value': np.float64(0.018114356791668697), 'U2_value': np.float64(0.07947490669120306), 'S_value': np.float64(0.19250119449468417), 'MI_value': np.float64(0.49789110131486575)},
       {'feature_pair': ('Insulin - Regular', 'Glucose (serum)'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.20234711227717744), 'U1_value': np.float64(0.04748900255380732), 'U2_value': np.float64(0.015939102143994174), 'S_value': np.float64(0.19124422021009208), 'MI_value': np.float64(0.4570194371850717)},
       {'feature_pair': ('Insulin - Regular', 'Glucose (serum)'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.16435925045774968), 'U1_value': np.float64(0.17513718840556725), 'U2_value': np.float64(1.009634525844082e-07), 'S_value': np.float64(0.1696525192583821), 'MI_value': np.float64(0.5091490590851517)},
       {'feature_pair': ('Insulin

In [14]:
mimiciv_rus_results[12:18]

array([{'feature_pair': ('Insulin - Regular', 'Potassium (serum)'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.20278101838852425), 'U1_value': np.float64(0.023133962041346618), 'U2_value': np.float64(0.18134933711645404), 'S_value': np.float64(0.08448422588981686), 'MI_value': np.float64(0.49174854343614094)},
       {'feature_pair': ('Insulin - Regular', 'Potassium (serum)'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.20089035777235975), 'U1_value': np.float64(0.04894559849013522), 'U2_value': np.float64(0.05528630165606725), 'S_value': np.float64(0.1383504538129363), 'MI_value': np.float64(0.44347271173150005)},
       {'feature_pair': ('Insulin - Regular', 'Potassium (serum)'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.16094476023035215), 'U1_value': np.float64(0.1785516425059096), 'U2_value': np.float64(5.744881545844255e-08), 'S_value': np.float64(0.14690267960596465), 'MI_value': np.float64(0.4863991397910409)},
       {'feature_pair': ('In

In [15]:
mimiciv_rus_results[18:24]

array([{'feature_pair': ('Insulin - Regular', 'Magnesium'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.16755070657927576), 'U1_value': np.float64(0.05836427550114074), 'U2_value': np.float64(1.8442526671435272e-07), 'S_value': np.float64(0.03969163521191307), 'MI_value': np.float64(0.26560680171759715)},
       {'feature_pair': ('Insulin - Regular', 'Magnesium'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.16517796545700103), 'U1_value': np.float64(0.08465812755317068), 'U2_value': np.float64(1.2343090410782842e-07), 'S_value': np.float64(0.03846816296854072), 'MI_value': np.float64(0.28830437940961695)},
       {'feature_pair': ('Insulin - Regular', 'Magnesium'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.16265353566830998), 'U1_value': np.float64(0.17684275720641437), 'U2_value': np.float64(1.2527558371478833e-07), 'S_value': np.float64(0.04009844069914836), 'MI_value': np.float64(0.37959485884945643)},
       {'feature_pair': ('Insulin - Regular

In [16]:
mimiciv_rus_results[24:30]

array([{'feature_pair': ('Insulin - Regular', 'BUN'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.18537168706120588), 'U1_value': np.float64(0.04054339271419252), 'U2_value': np.float64(0.022594158447576892), 'S_value': np.float64(0.20198667527433722), 'MI_value': np.float64(0.45049591349731244)},
       {'feature_pair': ('Insulin - Regular', 'BUN'), 'lag': 2, 'dominant_term': 'U1', 'R_value': np.float64(0.11774765298690099), 'U1_value': np.float64(0.13208843801282497), 'U2_value': np.float64(6.772380083429353e-08), 'S_value': np.float64(0.15911101653231313), 'MI_value': np.float64(0.4089471752558391)},
       {'feature_pair': ('Insulin - Regular', 'BUN'), 'lag': 3, 'dominant_term': 'U1', 'R_value': np.float64(0.056294842479980345), 'U1_value': np.float64(0.28320160352118423), 'U2_value': np.float64(4.6275105604772246e-08), 'S_value': np.float64(0.12088360543139881), 'MI_value': np.float64(0.4603800977076694)},
       {'feature_pair': ('Insulin - Regular', 'BUN'), 'lag': 4,

In [17]:
mimiciv_rus_results[30:36]

array([{'feature_pair': ('Furosemide (Lasix) 250/50', 'Glucose (serum)'), 'lag': 1, 'dominant_term': 'U2', 'R_value': np.float64(0.05823218707137051), 'U1_value': np.float64(5.164196877488814e-09), 'U2_value': np.float64(0.22904321742169995), 'S_value': np.float64(1.782501396485614e-08), 'MI_value': np.float64(0.28727542748228146)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Glucose (serum)'), 'lag': 2, 'dominant_term': 'U2', 'R_value': np.float64(0.04292271283075921), 'U1_value': np.float64(5.295312344480277e-09), 'U2_value': np.float64(0.17536343293675913), 'S_value': np.float64(1.9728246120065407e-08), 'MI_value': np.float64(0.21828617079107696)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Glucose (serum)'), 'lag': 3, 'dominant_term': 'U2', 'R_value': np.float64(0.029770751078713048), 'U1_value': np.float64(4.2633834617302645e-09), 'U2_value': np.float64(0.13458857371108537), 'S_value': np.float64(1.3705570678368417e-08), 'MI_value': np.float64(0.164359342758

In [18]:
mimiciv_rus_results[36:42]

array([{'feature_pair': ('Furosemide (Lasix) 250/50', 'Potassium (serum)'), 'lag': 1, 'dominant_term': 'U2', 'R_value': np.float64(0.058232191140698), 'U1_value': np.float64(3.1894620387739668e-12), 'U2_value': np.float64(0.32589796156171297), 'S_value': np.float64(0.008484392625600246), 'MI_value': np.float64(0.3926145453312005)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Potassium (serum)'), 'lag': 2, 'dominant_term': 'U2', 'R_value': np.float64(0.0429227162671586), 'U1_value': np.float64(2.159234539801525e-11), 'U2_value': np.float64(0.21325377282834412), 'S_value': np.float64(0.016773023170265577), 'MI_value': np.float64(0.27294951228736086)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Potassium (serum)'), 'lag': 3, 'dominant_term': 'U2', 'R_value': np.float64(0.029770755209792), 'U1_value': np.float64(3.871608071016679e-11), 'U2_value': np.float64(0.13117402820357163), 'S_value': np.float64(0.02497021136693811), 'MI_value': np.float64(0.185914994819018)},


In [19]:
mimiciv_rus_results[42:48]

array([{'feature_pair': ('Furosemide (Lasix) 250/50', 'Magnesium'), 'lag': 1, 'dominant_term': 'U2', 'R_value': np.float64(0.055124554368533224), 'U1_value': np.float64(0.0031076374057697653), 'U2_value': np.float64(0.1124262780385782), 'S_value': np.float64(0.12490987724540878), 'MI_value': np.float64(0.29556834705828977)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Magnesium'), 'lag': 2, 'dominant_term': 'U2', 'R_value': np.float64(0.04214132040332888), 'U1_value': np.float64(0.0007813950302704558), 'U2_value': np.float64(0.12303673052918748), 'S_value': np.float64(0.08578375041920419), 'MI_value': np.float64(0.25174319638199116)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'Magnesium'), 'lag': 3, 'dominant_term': 'U2', 'R_value': np.float64(0.029770749411212855), 'U1_value': np.float64(4.165177191962838e-09), 'U2_value': np.float64(0.13288288599832654), 'S_value': np.float64(0.05508655855206601), 'MI_value': np.float64(0.21774019812678233)},
       {'feature_p

In [20]:
mimiciv_rus_results[48:54]

array([{'feature_pair': ('Furosemide (Lasix) 250/50', 'BUN'), 'lag': 1, 'dominant_term': 'U2', 'R_value': np.float64(0.058232187927324736), 'U1_value': np.float64(5.125675806442544e-09), 'U2_value': np.float64(0.1497334765201467), 'S_value': np.float64(0.1939857829054322), 'MI_value': np.float64(0.40195145247857966)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'BUN'), 'lag': 2, 'dominant_term': 'S', 'R_value': np.float64(0.04292271306980992), 'U1_value': np.float64(5.935605754676513e-09), 'U2_value': np.float64(0.07482496082052038), 'S_value': np.float64(0.17475630292192157), 'MI_value': np.float64(0.2925039827478575)},
       {'feature_pair': ('Furosemide (Lasix) 250/50', 'BUN'), 'lag': 3, 'dominant_term': 'S', 'R_value': np.float64(0.029770751888562895), 'U1_value': np.float64(4.2381455691073805e-09), 'U2_value': np.float64(0.026524106729048508), 'S_value': np.float64(0.16039851202660366), 'MI_value': np.float64(0.2166933748823609)},
       {'feature_pair': ('Furosemide (L

In [21]:
mimiciv_rus_results[54:60]

array([{'feature_pair': ('Glucose (serum)', 'Potassium (serum)'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.27818507702305717), 'U1_value': np.float64(0.009090306048852324), 'U2_value': np.float64(0.10594508712153186), 'S_value': np.float64(-3.426292644048701e-08), 'MI_value': np.float64(0.39322043593051453)},
       {'feature_pair': ('Glucose (serum)', 'Potassium (serum)'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.20021405225962374), 'U1_value': np.float64(0.01807208461547564), 'U2_value': np.float64(0.055962447238467494), 'S_value': np.float64(-3.2366104463221745e-08), 'MI_value': np.float64(0.2742485517474621)},
       {'feature_pair': ('Glucose (serum)', 'Potassium (serum)'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.13729284887687765), 'U1_value': np.float64(0.027066446708354626), 'U2_value': np.float64(0.023651946429922965), 'S_value': np.float64(-3.561276112895584e-08), 'MI_value': np.float64(0.18801120640239422)},
       {'feature_pair

In [22]:
mimiciv_rus_results[60:66]

array([{'feature_pair': ('Glucose (serum)', 'Magnesium'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.15145556647699537), 'U1_value': np.float64(0.13581985083902798), 'U2_value': np.float64(0.01609529363035023), 'S_value': np.float64(0.022447912051701435), 'MI_value': np.float64(0.32581862299807485)},
       {'feature_pair': ('Glucose (serum)', 'Magnesium'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.13110112176177413), 'U1_value': np.float64(0.0871850373284415), 'U2_value': np.float64(0.03407692849643268), 'S_value': np.float64(0.014608341453498652), 'MI_value': np.float64(0.2669714290401469)},
       {'feature_pair': ('Glucose (serum)', 'Magnesium'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.10927274958501398), 'U1_value': np.float64(0.05508657734760589), 'U2_value': np.float64(0.05338089070253391), 'S_value': np.float64(0.006400774229169348), 'MI_value': np.float64(0.22414099186432318)},
       {'feature_pair': ('Glucose (serum)', 'Magnesium'),

In [23]:
mimiciv_rus_results[66:72]

array([{'feature_pair': ('Glucose (serum)', 'BUN'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.20551051031635975), 'U1_value': np.float64(0.08176487501991478), 'U2_value': np.float64(0.002455171955120792), 'S_value': np.float64(0.11238372529315122), 'MI_value': np.float64(0.4021142825845466)},
       {'feature_pair': ('Glucose (serum)', 'BUN'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.11774767214466048), 'U1_value': np.float64(0.10053848650516868), 'U2_value': np.float64(4.3909998291502544e-09), 'S_value': np.float64(0.0745747359673451), 'MI_value': np.float64(0.2928608990081742)},
       {'feature_pair': ('Glucose (serum)', 'BUN'), 'lag': 3, 'dominant_term': 'U1', 'R_value': np.float64(0.05629485710514498), 'U1_value': np.float64(0.10806447159998525), 'U2_value': np.float64(4.7809498077006425e-09), 'S_value': np.float64(0.05292477174384008), 'MI_value': np.float64(0.21728410522992014)},
       {'feature_pair': ('Glucose (serum)', 'BUN'), 'lag': 4, 'dominant

In [24]:
mimiciv_rus_results[72:78]

array([{'feature_pair': ('Potassium (serum)', 'Magnesium'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.14566981568983284), 'U1_value': np.float64(0.2384603368458455), 'U2_value': np.float64(0.02188103973394488), 'S_value': np.float64(1.1515392750460762e-08), 'MI_value': np.float64(0.4060112037850161)},
       {'feature_pair': ('Potassium (serum)', 'Magnesium'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.12053520421216612), 'U1_value': np.float64(0.13564128561137598), 'U2_value': np.float64(0.04464282920489354), 'S_value': np.float64(1.9791749295006156e-08), 'MI_value': np.float64(0.3008193388201848)},
       {'feature_pair': ('Potassium (serum)', 'Magnesium'), 'lag': 3, 'dominant_term': 'R', 'R_value': np.float64(0.0942335120980285), 'U1_value': np.float64(0.06671127012666223), 'U2_value': np.float64(0.06842012144641364), 'S_value': np.float64(1.2658766368467411e-08), 'MI_value': np.float64(0.22936491632987105)},
       {'feature_pair': ('Potassium (serum)', '

In [25]:
mimiciv_rus_results[78:84]

array([{'feature_pair': ('Potassium (serum)', 'BUN'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.20796566577948378), 'U1_value': np.float64(0.17616448941809593), 'U2_value': np.float64(2.3106495511497717e-10), 'S_value': np.float64(0.008484389902555955), 'MI_value': np.float64(0.3926145453312005)},
       {'feature_pair': ('Potassium (serum)', 'BUN'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.11774767422499277), 'U1_value': np.float64(0.1384288154264121), 'U2_value': np.float64(2.477871306383441e-10), 'S_value': np.float64(0.018298370005465636), 'MI_value': np.float64(0.2744748599046575)},
       {'feature_pair': ('Potassium (serum)', 'BUN'), 'lag': 3, 'dominant_term': 'U1', 'R_value': np.float64(0.056294858788180996), 'U1_value': np.float64(0.1046499243930305), 'U2_value': np.float64(3.666990342535475e-10), 'S_value': np.float64(0.029813619207289865), 'MI_value': np.float64(0.19075840275520012)},
       {'feature_pair': ('Potassium (serum)', 'BUN'), 'lag': 4

In [26]:
mimiciv_rus_results[84:90]

array([{'feature_pair': ('Magnesium', 'BUN'), 'lag': 1, 'dominant_term': 'R', 'R_value': np.float64(0.13431971290380945), 'U1_value': np.float64(0.03323113897527597), 'U2_value': np.float64(0.07364595067830015), 'S_value': np.float64(0.16629867432997042), 'MI_value': np.float64(0.407495476887356)},
       {'feature_pair': ('Magnesium', 'BUN'), 'lag': 2, 'dominant_term': 'R', 'R_value': np.float64(0.09557229993579464), 'U1_value': np.float64(0.06960574135445573), 'U2_value': np.float64(0.02217537289051266), 'S_value': np.float64(0.11687966843326358), 'MI_value': np.float64(0.3042330826140268)},
       {'feature_pair': ('Magnesium', 'BUN'), 'lag': 3, 'dominant_term': 'U1', 'R_value': np.float64(0.05298357252373628), 'U1_value': np.float64(0.1096700699816697), 'U2_value': np.float64(0.0033112862732268497), 'S_value': np.float64(0.06940397968369316), 'MI_value': np.float64(0.23536890846232583)},
       {'feature_pair': ('Magnesium', 'BUN'), 'lag': 4, 'dominant_term': 'U1', 'R_value': np.fl